In [ ]:
import os, pickle

import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix

import AE as ae

import matplotlib.pyplot as plt

## Load Data

In [ ]:
# Load found event locations
directory = 'INSERT DIRECTORY' # Directory where event locations generated by AE Evt Finder.ipynb are saved.

with open(os.path.join(directory, 'INSERT FILENAME'), 'rb') as f:
    evt_df = pickle.load(f)

In [ ]:
directory = 'INSERT DIRECTORY' # Directory where events generated by Simulation.py are saved.

files = [] # List of simulation filenames

SNRs = [float(x.split('_')[0]) for x in files]

# Load Data and Ground Truths
raw_traces, ground_truths = ae.load_SNR_data(directory, files)

In [ ]:
# DataFrame of I(t) traces
raw_df = pd.Series(raw_traces).to_frame()
raw_df.columns = ['traces']
raw_df.index.names = ['SNR']
raw_df.head()

In [ ]:
# DataFrame of Ground Truths
truth_df = pd.Series(ground_truths).to_frame()
truth_df.columns = ['ground_truths']
truth_df.index.names = ['SNR']
truth_df.head()

## Generate Predictions
Convert the saved event locations to one hot encodings

In [ ]:
def my_func(row):
    traces = raw_df.loc[row.index[0][0]][0]
    
    preds = []
    for dev in row.iteritems():
        pred = ae.locs_to_preds(dev[1], traces)
        preds.append(pred)
    return preds

evt_df['predictions'] = evt_df.apply(my_func)

## Score Predictions

In [ ]:
# Create new DataFrame to store scores
columns = ['SNR', 'Thresh', 'trace_idx', 'FPR', 'TPR', 'FNR', 'TNR']
result_df = pd.DataFrame(columns=columns)
#result_df = result_df.set_index(['SNR', 'St_Dev'])
result_df.head()

In [ ]:
# Initialise window size and step size
window_size = 5
step_size = 5

# Check for missed trace points
untested = (1000000 - (window_size - step_size)) % step_size
untested

In [ ]:
for snr, thresh in list(evt_df.index.unique()):
    print('SNR: ' + str(snr) + ', Threshold: ' + str(thresh))
    preds = evt_df.loc[snr, thresh]['predictions'] # Get predictions
    truths = truth_df.loc[snr][0] # Get ground truth
    
    assert preds.shape == truths.shape
    for i in range(preds.shape[0]):
        trace_idx = i
        
        # Generate detection scores
        C = confusion_matrix(truths[i][:], preds[i][:], normalize='true')
        # C = confusion_matrix(truths[i][:-untested], preds[i][:-untested], normalize='true')
        tpr = C[1, 1]
        fpr = C[0, 1]
        tnr = C[0, 0]
        fnr = C[1, 0]
        
        # Populate DataFrame
        entry = dict(zip( columns, [[snr], [thresh], [trace_idx], [fpr], [tpr], [fnr], [tnr]]))                 
        entry_df = pd.DataFrame(entry)
        
        result_df = pd.concat([result_df, entry_df])

# Tidy DataFrame
result_df = result_df.set_index(['SNR', 'Thresh'])        
result_df.head()

# Save Results

In [ ]:
with open('INSERT FILENAME', 'wb') as f:
    pickle.dump(result_df, f)